[View in Colaboratory](https://colab.research.google.com/github/Jacksu20160407/Google_Colaboratory/blob/master/Test.ipynb)

In [0]:
!pip install pytest

In [0]:
!pip install ipdb

In [0]:
import os
import ipdb as pdb
import multiprocessing

import numpy as np
import pytest
from csv import reader
from csv import Sniffer
import shutil
from keras import optimizers
from keras import initializers
from keras import callbacks
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, add, dot, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D
from keras.utils.test_utils import get_test_data
from keras.utils.test_utils import keras_test
from keras import backend as K
from keras.utils import np_utils
try:
    from unittest.mock import patch
except:
    from mock import patch


In [0]:
input_dim = 2
num_hidden = 4
num_classes = 2
batch_size = 5
train_samples = 20
test_samples = 20

In [15]:

%prun
@keras_test
def test_TerminateOnNaN():
    np.random.seed(1337)
    (X_train, y_train), (X_test, y_test) = get_test_data(num_train=train_samples,
                                                         num_test=test_samples,
                                                         input_shape=(input_dim,),
                                                         classification=True,
                                                         num_classes=num_classes)
    pdb.set_trace()
    y_test = np_utils.to_categorical(y_test)
    y_train = np_utils.to_categorical(y_train)
    cbks = [callbacks.TerminateOnNaN()]
    model = Sequential()
    initializer = initializers.Constant(value=1e5)
    for _ in range(5):
        model.add(Dense(num_hidden, input_dim=input_dim, activation='relu',
                        kernel_initializer=initializer))
    model.add(Dense(num_classes, activation='linear'))
    model.compile(loss='mean_squared_error',
                  optimizer='rmsprop')

    # case 1 fit
    history = model.fit(X_train, y_train, batch_size=batch_size,
                        validation_data=(X_test, y_test), callbacks=cbks, epochs=20)
    loss = history.history['loss']
    assert len(loss) == 1
    assert loss[0] == np.inf

    # case 2 fit_generator
    def data_generator():
        max_batch_index = len(X_train) // batch_size
        i = 0
        while 1:
            yield (X_train[i * batch_size: (i + 1) * batch_size],
                   y_train[i * batch_size: (i + 1) * batch_size])
            i += 1
            i = i % max_batch_index
    history = model.fit_generator(data_generator(),
                                  len(X_train),
                                  validation_data=(X_test, y_test),
                                  callbacks=cbks,
                                  epochs=20)
    loss = history.history['loss']
    assert len(loss) == 1
    assert loss[0] == np.inf or np.isnan(loss[0])


In [0]:
import tensorflow as tf
import numpy as np
from numba import jit
from time import time, sleep
import pandas as pd
import json

In [0]:
class BenchMark:

    def __init__(self):
        conf = tf.ConfigProto()
        conf.gpu_options.per_process_gpu_memory_fraction = .95
        conf.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
        conf.gpu_options.allow_growth = True
        self.sess = tf.Session(config=conf)

        np.random.seed(0)
        self.tf_bboxes1 = tf.placeholder(dtype=tf.float16, shape=[None, 4])
        self.tf_bboxes2 = tf.placeholder(dtype=tf.float16, shape=[None, 4])

    def printer(self, to_run):
        print(self.sess.run(to_run))

    def get_2_bbxoes(self, num_boxes_in_1=10000, num_boxes_in_2=100000):
        """
        :param num_boxes_in_1: int, numbers of boxes; max 10000, limitation because of the GPU memory
        :param num_boxes_in_2: int, numbers of boxes; max 100000, limitation because of the GPU memory
        :return: boxes1, boxes2
        """
        # generating random co-ordinates of [x1,y1,x2,y2]
        boxes1 = np.reshape(np.random.randint(high=1200, low=0, size=num_boxes_in_1 * 4), newshape=(num_boxes_in_1, 4))
        boxes2 = np.reshape(np.random.randint(high=1200, low=0, size=num_boxes_in_2 * 4), newshape=(num_boxes_in_2, 4))
        return boxes1, boxes2

    def np_no_vec_no_jit_iou(self, boxes1, boxes2):
        """
        Calculate IOU between a bounding box and a set of bounding boxes.
        :param box: [x1,y1,x2,y2]
        :param boxes:[[x1,y1,x2,y2],[x1,y1,x2,y2],[x1,y1,x2,y2]]
        :return: corresponding IOU values
        """
        def run(box, boxes):
            ww = np.maximum(np.minimum(box[0] + box[2], boxes[:, 0] + boxes[:, 2]) -
                            np.maximum(box[0], boxes[:, 0]),
                            0)
            hh = np.maximum(np.minimum(box[1] + box[3], boxes[:, 1] + boxes[:, 3]) -
                            np.maximum(box[1], boxes[:, 1]),
                            0)
            uu = box[2] * box[3] + boxes[:, 2] * boxes[:, 3]
            return ww * hh / (uu - ww * hh)

        tic = time()
        for b in boxes1:
            run(b, boxes2)
        toc = time()
        return toc - tic

    def np_no_vec_jit_iou(self, boxes1, boxes2):
        """
        Calculate IOU between a bounding box and a set of bounding boxes.
        :param box: [x1,y1,x2,y2]
        :param boxes:[[x1,y1,x2,y2],[x1,y1,x2,y2],[x1,y1,x2,y2]]
        :return: corresponding IOU values
        """

        @jit(nopython=True)
        def run(box, boxes):
            ww = np.maximum(np.minimum(box[0] + box[2], boxes[:, 0] + boxes[:, 2]) -
                            np.maximum(box[0], boxes[:, 0]),
                            0)
            hh = np.maximum(np.minimum(box[1] + box[3], boxes[:, 1] + boxes[:, 3]) -
                            np.maximum(box[1], boxes[:, 1]),
                            0)
            uu = box[2] * box[3] + boxes[:, 2] * boxes[:, 3]
            return ww * hh / (uu - ww * hh)

        tic = time()
        for b in boxes1:
            run(b, boxes2)
        toc = time()
        return toc - tic

    def np_vec_no_jit_iou(self, boxes1, boxes2):
        def run(bboxes1, bboxes2):
            x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
            x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)

            # determine the (x, y)-coordinates of the intersection rectangle
            xA = np.maximum(x11, np.transpose(x21))
            yA = np.maximum(y11, np.transpose(y21))
            xB = np.minimum(x12, np.transpose(x22))
            yB = np.minimum(y12, np.transpose(y22))

            # compute the area of intersection rectangle
            interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)

            # compute the area of both the prediction and ground-truth rectangles
            boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
            boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)

            iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)

            return iou

        tic = time()
        run(boxes1, boxes2)
        toc = time()
        return toc - tic

    def np_vec_jit_iou(self, boxes1, boxes2):

        @jit
        def run(bboxes1, bboxes2):
            x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
            x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)

            # determine the (x, y)-coordinates of the intersection rectangle
            xA = np.maximum(x11, np.transpose(x21))
            yA = np.maximum(y11, np.transpose(y21))
            xB = np.minimum(x12, np.transpose(x22))
            yB = np.minimum(y12, np.transpose(y22))

            # compute the area of intersection rectangle
            interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)

            # compute the area of both the prediction and ground-truth rectangles
            boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
            boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)

            iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)

            return iou

        # %time np_vec_jit_iou(boxes1, boxes2)
        tic = time()
        run(boxes1, boxes2)
        toc = time()
        return toc - tic

    def tf_iou(self, boxes1, boxes2):
        def run(tb1, tb2):
            x11, y11, x12, y12 = tf.split(tb1, 4, axis=1)
            x21, y21, x22, y22 = tf.split(tb2, 4, axis=1)

            # determine the (x, y)-coordinates of the intersection rectangle
            xA = tf.maximum(x11, tf.transpose(x21))
            yA = tf.maximum(y11, tf.transpose(y21))
            xB = tf.minimum(x12, tf.transpose(x22))
            yB = tf.minimum(y12, tf.transpose(y22))

            # compute the area of intersection rectangle
            interArea = tf.maximum((xB - xA + 1), 0) * tf.maximum((yB - yA + 1), 0)

            # compute the area of both the prediction and ground-truth rectangles
            boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
            boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)

            iou = interArea / (boxAArea + tf.transpose(boxBArea) - interArea)

            return iou

        op = run(self.tf_bboxes1, self.tf_bboxes2)
        self.sess.run(op, feed_dict={self.tf_bboxes1: boxes1, self.tf_bboxes2: boxes2})
        tic = time()
        self.sess.run(op, feed_dict={self.tf_bboxes1: boxes1, self.tf_bboxes2: boxes2})
        toc = time()
        return toc - tic

    def benchmark(self, box1_max=10001, box1_step=5, box2_max=10001, box2_step=5):
        row = list()

        for num_boxes_in_1, num_boxes_in_2 in zip(range(1, box1_max, box1_step), range(1, box2_max, box2_step)):
            print("num_boxes_in_1: {}, \t num_boxes_in_2: {}".format(num_boxes_in_1, num_boxes_in_2))
            boxes1, boxes2 = self.get_2_bbxoes(num_boxes_in_1, num_boxes_in_2)

            each_row = dict()
            each_row.update({"box_A_size": num_boxes_in_1,
                             "box_B_size": num_boxes_in_2,
                             "1_np_no_vec_no_jit_iou": self.np_no_vec_no_jit_iou(boxes1, boxes2),
                             "2_np_no_vec_jit_iou": self.np_no_vec_jit_iou(boxes1, boxes2),
                             "3_np_vec_no_jit_iou": self.np_vec_no_jit_iou(boxes1, boxes2),
                             "4_np_vec_jit_iou": self.np_vec_jit_iou(boxes1, boxes2),
                             "5_tf_iou": self.tf_iou(boxes1, boxes2)
                             })
            row.append(each_row)

        return row

In [0]:
analysis = BenchMark().benchmark()
df = pd.DataFrame(data=analysis)
df.to_csv("./analysis.csv", index=False)

num_boxes_in_1: 1, 	 num_boxes_in_2: 1
num_boxes_in_1: 6, 	 num_boxes_in_2: 6
num_boxes_in_1: 11, 	 num_boxes_in_2: 11
num_boxes_in_1: 16, 	 num_boxes_in_2: 16
num_boxes_in_1: 21, 	 num_boxes_in_2: 21
num_boxes_in_1: 26, 	 num_boxes_in_2: 26
num_boxes_in_1: 31, 	 num_boxes_in_2: 31
num_boxes_in_1: 36, 	 num_boxes_in_2: 36
num_boxes_in_1: 41, 	 num_boxes_in_2: 41
num_boxes_in_1: 46, 	 num_boxes_in_2: 46
num_boxes_in_1: 51, 	 num_boxes_in_2: 51
num_boxes_in_1: 56, 	 num_boxes_in_2: 56
num_boxes_in_1: 61, 	 num_boxes_in_2: 61
num_boxes_in_1: 66, 	 num_boxes_in_2: 66
num_boxes_in_1: 71, 	 num_boxes_in_2: 71
num_boxes_in_1: 76, 	 num_boxes_in_2: 76
num_boxes_in_1: 81, 	 num_boxes_in_2: 81
num_boxes_in_1: 86, 	 num_boxes_in_2: 86
num_boxes_in_1: 91, 	 num_boxes_in_2: 91
num_boxes_in_1: 96, 	 num_boxes_in_2: 96
num_boxes_in_1: 101, 	 num_boxes_in_2: 101
num_boxes_in_1: 106, 	 num_boxes_in_2: 106


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:94: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:129: RuntimeWarning: divide by zero encountered in divide


num_boxes_in_1: 111, 	 num_boxes_in_2: 111
num_boxes_in_1: 116, 	 num_boxes_in_2: 116
num_boxes_in_1: 121, 	 num_boxes_in_2: 121


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in divide


num_boxes_in_1: 126, 	 num_boxes_in_2: 126
num_boxes_in_1: 131, 	 num_boxes_in_2: 131
num_boxes_in_1: 136, 	 num_boxes_in_2: 136
num_boxes_in_1: 141, 	 num_boxes_in_2: 141
num_boxes_in_1: 146, 	 num_boxes_in_2: 146
num_boxes_in_1: 151, 	 num_boxes_in_2: 151
num_boxes_in_1: 156, 	 num_boxes_in_2: 156
num_boxes_in_1: 161, 	 num_boxes_in_2: 161
num_boxes_in_1: 166, 	 num_boxes_in_2: 166
num_boxes_in_1: 171, 	 num_boxes_in_2: 171
num_boxes_in_1: 176, 	 num_boxes_in_2: 176
num_boxes_in_1: 181, 	 num_boxes_in_2: 181
num_boxes_in_1: 186, 	 num_boxes_in_2: 186
num_boxes_in_1: 191, 	 num_boxes_in_2: 191
num_boxes_in_1: 196, 	 num_boxes_in_2: 196
num_boxes_in_1: 201, 	 num_boxes_in_2: 201
num_boxes_in_1: 206, 	 num_boxes_in_2: 206
num_boxes_in_1: 211, 	 num_boxes_in_2: 211
num_boxes_in_1: 216, 	 num_boxes_in_2: 216
num_boxes_in_1: 221, 	 num_boxes_in_2: 221
num_boxes_in_1: 226, 	 num_boxes_in_2: 226
num_boxes_in_1: 231, 	 num_boxes_in_2: 231
num_boxes_in_1: 236, 	 num_boxes_in_2: 236
num_boxes_i

...

num_boxes_in_1: 2781, 	 num_boxes_in_2: 2781
num_boxes_in_1: 2786, 	 num_boxes_in_2: 2786
num_boxes_in_1: 2791, 	 num_boxes_in_2: 2791
num_boxes_in_1: 2796, 	 num_boxes_in_2: 2796
num_boxes_in_1: 2801, 	 num_boxes_in_2: 2801
num_boxes_in_1: 2806, 	 num_boxes_in_2: 2806
num_boxes_in_1: 2811, 	 num_boxes_in_2: 2811
num_boxes_in_1: 2816, 	 num_boxes_in_2: 2816
num_boxes_in_1: 2821, 	 num_boxes_in_2: 2821
num_boxes_in_1: 2826, 	 num_boxes_in_2: 2826
num_boxes_in_1: 2831, 	 num_boxes_in_2: 2831
num_boxes_in_1: 2836, 	 num_boxes_in_2: 2836
num_boxes_in_1: 2841, 	 num_boxes_in_2: 2841
num_boxes_in_1: 2846, 	 num_boxes_in_2: 2846
num_boxes_in_1: 2851, 	 num_boxes_in_2: 2851
num_boxes_in_1: 2856, 	 num_boxes_in_2: 2856
num_boxes_in_1: 2861, 	 num_boxes_in_2: 2861
num_boxes_in_1: 2866, 	 num_boxes_in_2: 2866
num_boxes_in_1: 2871, 	 num_boxes_in_2: 2871
num_boxes_in_1: 2876, 	 num_boxes_in_2: 2876
num_boxes_in_1: 2881, 	 num_boxes_in_2: 2881
num_boxes_in_1: 2886, 	 num_boxes_in_2: 2886
num_boxes_

num_boxes_in_1: 3111, 	 num_boxes_in_2: 3111
num_boxes_in_1: 3116, 	 num_boxes_in_2: 3116
num_boxes_in_1: 3121, 	 num_boxes_in_2: 3121
num_boxes_in_1: 3126, 	 num_boxes_in_2: 3126
num_boxes_in_1: 3131, 	 num_boxes_in_2: 3131
num_boxes_in_1: 3136, 	 num_boxes_in_2: 3136
num_boxes_in_1: 3141, 	 num_boxes_in_2: 3141
num_boxes_in_1: 3146, 	 num_boxes_in_2: 3146
num_boxes_in_1: 3151, 	 num_boxes_in_2: 3151
num_boxes_in_1: 3156, 	 num_boxes_in_2: 3156
num_boxes_in_1: 3161, 	 num_boxes_in_2: 3161
num_boxes_in_1: 3166, 	 num_boxes_in_2: 3166
num_boxes_in_1: 3171, 	 num_boxes_in_2: 3171
num_boxes_in_1: 3176, 	 num_boxes_in_2: 3176
num_boxes_in_1: 3181, 	 num_boxes_in_2: 3181
num_boxes_in_1: 3186, 	 num_boxes_in_2: 3186
num_boxes_in_1: 3191, 	 num_boxes_in_2: 3191
num_boxes_in_1: 3196, 	 num_boxes_in_2: 3196
num_boxes_in_1: 3201, 	 num_boxes_in_2: 3201
num_boxes_in_1: 3206, 	 num_boxes_in_2: 3206
num_boxes_in_1: 3211, 	 num_boxes_in_2: 3211
num_boxes_in_1: 3216, 	 num_boxes_in_2: 3216
num_boxes_

num_boxes_in_1: 3441, 	 num_boxes_in_2: 3441
num_boxes_in_1: 3446, 	 num_boxes_in_2: 3446
num_boxes_in_1: 3451, 	 num_boxes_in_2: 3451
num_boxes_in_1: 3456, 	 num_boxes_in_2: 3456
num_boxes_in_1: 3461, 	 num_boxes_in_2: 3461
num_boxes_in_1: 3466, 	 num_boxes_in_2: 3466
num_boxes_in_1: 3471, 	 num_boxes_in_2: 3471
num_boxes_in_1: 3476, 	 num_boxes_in_2: 3476
num_boxes_in_1: 3481, 	 num_boxes_in_2: 3481
num_boxes_in_1: 3486, 	 num_boxes_in_2: 3486
num_boxes_in_1: 3491, 	 num_boxes_in_2: 3491
num_boxes_in_1: 3496, 	 num_boxes_in_2: 3496
num_boxes_in_1: 3501, 	 num_boxes_in_2: 3501
num_boxes_in_1: 3506, 	 num_boxes_in_2: 3506
num_boxes_in_1: 3511, 	 num_boxes_in_2: 3511
num_boxes_in_1: 3516, 	 num_boxes_in_2: 3516
num_boxes_in_1: 3521, 	 num_boxes_in_2: 3521
num_boxes_in_1: 3526, 	 num_boxes_in_2: 3526
num_boxes_in_1: 3531, 	 num_boxes_in_2: 3531
num_boxes_in_1: 3536, 	 num_boxes_in_2: 3536
num_boxes_in_1: 3541, 	 num_boxes_in_2: 3541
num_boxes_in_1: 3546, 	 num_boxes_in_2: 3546
num_boxes_

num_boxes_in_1: 3771, 	 num_boxes_in_2: 3771
num_boxes_in_1: 3776, 	 num_boxes_in_2: 3776
num_boxes_in_1: 3781, 	 num_boxes_in_2: 3781
num_boxes_in_1: 3786, 	 num_boxes_in_2: 3786
num_boxes_in_1: 3791, 	 num_boxes_in_2: 3791
num_boxes_in_1: 3796, 	 num_boxes_in_2: 3796
num_boxes_in_1: 3801, 	 num_boxes_in_2: 3801
num_boxes_in_1: 3806, 	 num_boxes_in_2: 3806
num_boxes_in_1: 3811, 	 num_boxes_in_2: 3811
num_boxes_in_1: 3816, 	 num_boxes_in_2: 3816
num_boxes_in_1: 3821, 	 num_boxes_in_2: 3821
num_boxes_in_1: 3826, 	 num_boxes_in_2: 3826
num_boxes_in_1: 3831, 	 num_boxes_in_2: 3831
num_boxes_in_1: 3836, 	 num_boxes_in_2: 3836
num_boxes_in_1: 3841, 	 num_boxes_in_2: 3841
num_boxes_in_1: 3846, 	 num_boxes_in_2: 3846
num_boxes_in_1: 3851, 	 num_boxes_in_2: 3851
num_boxes_in_1: 3856, 	 num_boxes_in_2: 3856
num_boxes_in_1: 3861, 	 num_boxes_in_2: 3861
num_boxes_in_1: 3866, 	 num_boxes_in_2: 3866
num_boxes_in_1: 3871, 	 num_boxes_in_2: 3871
num_boxes_in_1: 3876, 	 num_boxes_in_2: 3876
num_boxes_

num_boxes_in_1: 4101, 	 num_boxes_in_2: 4101
num_boxes_in_1: 4106, 	 num_boxes_in_2: 4106
num_boxes_in_1: 4111, 	 num_boxes_in_2: 4111
num_boxes_in_1: 4116, 	 num_boxes_in_2: 4116
num_boxes_in_1: 4121, 	 num_boxes_in_2: 4121
num_boxes_in_1: 4126, 	 num_boxes_in_2: 4126
num_boxes_in_1: 4131, 	 num_boxes_in_2: 4131
num_boxes_in_1: 4136, 	 num_boxes_in_2: 4136
num_boxes_in_1: 4141, 	 num_boxes_in_2: 4141
num_boxes_in_1: 4146, 	 num_boxes_in_2: 4146
num_boxes_in_1: 4151, 	 num_boxes_in_2: 4151
num_boxes_in_1: 4156, 	 num_boxes_in_2: 4156
num_boxes_in_1: 4161, 	 num_boxes_in_2: 4161
num_boxes_in_1: 4166, 	 num_boxes_in_2: 4166
num_boxes_in_1: 4171, 	 num_boxes_in_2: 4171
num_boxes_in_1: 4176, 	 num_boxes_in_2: 4176
num_boxes_in_1: 4181, 	 num_boxes_in_2: 4181
num_boxes_in_1: 4186, 	 num_boxes_in_2: 4186


In [0]:
!mkdir -p data

In [5]:
ls


data/  datalab/
